<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/notebooks/29_Optimizer_Internals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --------------------------------------------------------------
# ☁️ COLAB SETUP (Automatyczna instalacja środowiska)
# --------------------------------------------------------------
import sys
import os

# Sprawdzamy, czy jesteśmy w Google Colab
if 'google.colab' in sys.modules:
    print('☁️ Wykryto środowisko Google Colab. Konfiguruję...')

    # 1. Pobieramy plik requirements.txt bezpośrednio z repozytorium
    !wget -q https://raw.githubusercontent.com/takzen/ai-engineering-handbook/main/requirements.txt -O requirements.txt

    # 2. Instalujemy biblioteki
    print('⏳ Instaluję zależności (to może chwilę potrwać)...')
    !pip install -q -r requirements.txt

    print('✅ Gotowe! Środowisko jest zgodne z repozytorium.')
else:
    print('💻 Wykryto środowisko lokalne. Zakładam, że masz już uv/venv.')


# 🥋 Lekcja 29: Wnętrze Optymalizatora (Pisanie własnego SGD)

Optymalizator w PyTorch to klasa, która:
1.  Przechowuje listę parametrów (`params`) do aktualizacji.
2.  Przechowuje stan wewnętrzny (`state`) - np. pęd (Momentum) dla każdego parametru.
3.  W metodzie `step()` wykonuje matematykę: $W_{new} = W_{old} - lr \cdot \nabla W$.

**Kluczowa zasada:**
Optymalizator modyfikuje wagi **In-Place** (bez tworzenia nowych tensorów) i **bez gradientów** (używając `torch.no_grad()`).

Zbudujemy własną klasę `MySGD`.

In [1]:
import torch
import torch.nn as nn

# Prosty problem do rozwiązania: y = 2x
# Chcemy znaleźć wagę 2.0
X = torch.tensor([1.0, 2.0, 3.0, 4.0])
Y = torch.tensor([2.0, 4.0, 6.0, 8.0])

model = nn.Linear(1, 1, bias=False)
model.weight.data.fill_(0.0) # Startujemy od zera

print(f"Startowa waga: {model.weight.item()}")

Startowa waga: 0.0


## Implementacja `MySGD`

Musimy dziedziczyć po `torch.optim.Optimizer`.
Wymaga to zdefiniowania metody `step()`.

Zwróć uwagę na pętlę `for group in self.param_groups`.
To dlatego możemy napisać:
`optim.SGD([{'params': model.fc1.parameters(), 'lr': 0.1}, {'params': rest, 'lr': 0.01}])`
Optymalizator traktuje każdą grupę osobno.

In [2]:
from torch.optim import Optimizer

class MySGD(Optimizer):
    def __init__(self, params, lr=0.01):
        defaults = dict(lr=lr)
        super().__init__(params, defaults)

    def step(self):
        # Pętla po grupach parametrów (zazwyczaj jest jedna, ale może być więcej)
        for group in self.param_groups:
            lr = group['lr']
            
            # Pętla po parametrach w grupie (Wagi, Biasy)
            for p in group['params']:
                if p.grad is None:
                    continue
                
                # --- TUTAJ DZIEJE SIĘ MAGIA ---
                # Modyfikujemy wagę w oparciu o gradient.
                # Musimy użyć no_grad(), żeby ta operacja nie trafiła do grafu!
                with torch.no_grad():
                    # W = W - lr * grad
                    p.sub_(lr * p.grad)
                    
        return None

# Inicjalizacja naszego optymalizatora
optimizer = MySGD(model.parameters(), lr=0.1)
print("Własny SGD gotowy.")

Własny SGD gotowy.


## Test: Czy to działa?

Uruchomimy standardową pętlę treningową.
Jeśli waga `model.weight` zacznie zbliżać się do 2.0, to znaczy, że nasz kod działa.

In [3]:
criterion = nn.MSELoss()

print(f"Waga przed: {model.weight.item():.4f}")

for epoch in range(10):
    # 1. Forward
    # Musimy zmienić kształt X na [N, 1] bo Linear tego oczekuje
    pred = model(X.unsqueeze(1))
    
    # 2. Loss
    loss = criterion(pred, Y.unsqueeze(1))
    
    # 3. Zero Grad (To też metoda Optimizera, ale dziedziczymy ją z klasy bazowej)
    optimizer.zero_grad()
    
    # 4. Backward (Tu PyTorch liczy .grad dla każdego parametru)
    loss.backward()
    
    # 5. Step (Tu nasz MySGD odejmuje gradient od wagi)
    optimizer.step()
    
    print(f"Epoka {epoch+1}: Loss={loss.item():.4f}, Waga={model.weight.item():.4f}")

print("✅ Sukces! Waga dąży do 2.0.")

Waga przed: 0.0000
Epoka 1: Loss=30.0000, Waga=3.0000
Epoka 2: Loss=7.5000, Waga=1.5000
Epoka 3: Loss=1.8750, Waga=2.2500
Epoka 4: Loss=0.4688, Waga=1.8750
Epoka 5: Loss=0.1172, Waga=2.0625
Epoka 6: Loss=0.0293, Waga=1.9688
Epoka 7: Loss=0.0073, Waga=2.0156
Epoka 8: Loss=0.0018, Waga=1.9922
Epoka 9: Loss=0.0005, Waga=2.0039
Epoka 10: Loss=0.0001, Waga=1.9980
✅ Sukces! Waga dąży do 2.0.


## Level Up: Dodajemy Momentum (Pęd)

Zwykły SGD utyka w lokalnych dołkach.
Momentum działa jak ciężka kulka. Jeśli gradient mówi "Stop", kulka toczy się dalej siłą rozpędu.

Wzór:
$$ v_{t} = \mu \cdot v_{t-1} + g_{t} $$
$$ w_{t} = w_{t-1} - lr \cdot v_{t} $$

Musimy użyć słownika `self.state`, żeby zapamiętać prędkość ($v$) dla każdego parametru.

In [4]:
class MyMomentumSGD(Optimizer):
    def __init__(self, params, lr=0.01, momentum=0.9):
        defaults = dict(lr=lr, momentum=momentum)
        super().__init__(params, defaults)

    def step(self):
        for group in self.param_groups:
            lr = group['lr']
            mu = group['momentum']
            
            for p in group['params']:
                if p.grad is None:
                    continue
                
                # Pobieramy stan dla tego konkretnego parametru
                state = self.state[p]
                
                # Jeśli to pierwszy krok, inicjalizujemy bufor prędkości zerami
                if 'velocity' not in state:
                    state['velocity'] = torch.zeros_like(p.data)
                
                buf = state['velocity']
                
                with torch.no_grad():
                    # 1. Aktualizujemy prędkość: v = mu * v + grad
                    buf.mul_(mu).add_(p.grad)
                    
                    # 2. Aktualizujemy wagę: w = w - lr * v
                    p.sub_(lr * buf)

# Resetujemy model i testujemy Momentum
model.weight.data.fill_(0.0)
opt_mom = MyMomentumSGD(model.parameters(), lr=0.1, momentum=0.9)

print("\n--- TEST MOMENTUM ---")
# Zrobimy tylko 2 kroki, żeby zobaczyć "rozpędzanie"
for i in range(2):
    opt_mom.zero_grad()
    loss = criterion(model(X.unsqueeze(1)), Y.unsqueeze(1))
    loss.backward()
    opt_mom.step()
    print(f"Krok {i+1}: Waga={model.weight.item():.4f}")

# Zauważ: W kroku 2 zmiana powinna być WIĘKSZA niż w kroku 1 (bo nabraliśmy pędu!)


--- TEST MOMENTUM ---
Krok 1: Waga=3.0000
Krok 2: Waga=4.2000


## 🥋 Black Belt Summary

1.  **Optymalizator jest "głupi":** On nie wie, co to jest sieć neuronowa. On po prostu dostaje listę tensorów i odejmuje od nich ich `.grad`.
2.  **Stan (`state`):** To tutaj `Adam` trzyma swoje średnie ruchome (m i v), a `SGD` trzyma pęd. To dlatego optymalizatory zajmują pamięć VRAM! (Adam zużywa 2x więcej pamięci na parametry niż SGD).
3.  **In-place:** Wszystkie operacje wewnątrz `step()` muszą być in-place (`sub_`, `add_`) i w bloku `no_grad()`.